In [1]:
from model_service import ModelService


[WARNING 11-10 14:20:10] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [2]:
gcn_multi = ModelService(multi_model=True)
gcn_multi.load_models()

/home/papafrita/projects/soluGNN/solugnn/model_service.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.models = [torch.load(model_name) for model_name in model_nam

5 models found and loaded.


In [3]:
gcn_single = ModelService()
gcn_single.load_model('optimized_gcn_seed_7.pt')

Model optimized_gcn_seed_7.pt loaded.


In [4]:
from collection import retrieve_dataset, get_dataloader

In [5]:
test_set = retrieve_dataset(dataset_path='~/projects/soluGNN/data/test.csv')
test_loader = get_dataloader(test_set)


In [49]:
from model import initialize_optimizer, initialize_standardizer, test_model, initialize_loss, get_outputs

outputs = get_outputs(retrieve_dataset())

test_model(gcn_single.model,
           test_dataloader=test_loader,
           standardizer=initialize_standardizer(outputs),
           loss_fn=initialize_loss(),
           )



(tensor(0.1577, device='cuda:0', grad_fn=<DivBackward0>),
 1.0291787385940552,
 1.5381962)

In [50]:
import torch
from collection import Graph

standardizer=initialize_standardizer(outputs)
def get_pred(smile, standardizer=standardizer):
    mol_graph = Graph(smile)
    node_mat = mol_graph.node_mat
    adj_mat = mol_graph.adj_mat
    max_atoms = mol_graph.max_atoms
    node_vec_len = mol_graph.node_vec_len

    # Reshape inputs
    first_dim = int((torch.numel(torch.Tensor(node_mat)) / (max_atoms * node_vec_len)))
    node_mat = node_mat.reshape(first_dim, max_atoms, node_vec_len)
    adj_mat = adj_mat.reshape(first_dim, max_atoms, max_atoms)
    # Package inputs; send to GPU
    nn_input = (torch.Tensor(node_mat).cuda(), torch.Tensor(adj_mat).cuda())
    # Predict

    output = gcn_single.model(*nn_input)
    output = standardizer.restore(output.detach().cpu())
    return output.item()

In [51]:
import pandas as pd

test_data = pd.read_csv('../data/test.csv')
test_data.head()

,Unnamed: 0,iupac,smiles,expt,calc
0,178,1-pyrrolidin-1-ylethanone,CC(=O)N1CCCC1,-9.80,-7.831
1,466,propanenitrile,CCC#N,-3.84,-2.491
2,565,1-bromo-pentane,CCCCCBr,-0.10,0.824
3,51,2-methylbutan-1-ol,CC[C@@H](C)CO,-4.42,-2.995
4,34,N-ethylethanamine,CCNCC,-4.07,-2.986


In [52]:
test_data['predicted'] = test_data['smiles'].apply(get_pred)
test_data.head()

,Unnamed: 0,iupac,smiles,expt,calc,predicted
0,178,1-pyrrolidin-1-ylethanone,CC(=O)N1CCCC1,-9.80,-7.831,-4.719940
1,466,propanenitrile,CCC#N,-3.84,-2.491,-3.776811
2,565,1-bromo-pentane,CCCCCBr,-0.10,0.824,0.723713
3,51,2-methylbutan-1-ol,CC[C@@H](C)CO,-4.42,-2.995,-4.148237
4,34,N-ethylethanamine,CCNCC,-4.07,-2.986,-4.751862


In [53]:
from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(test_data['expt'], test_data['predicted'])

1.5381961635577326